Some basic tests are performed here with the aim of doing gradual learning as the season goes on using either 
model.fit() [retrain every period]
model.partialfit() [remember old training and train only on new games; this is available only for some models; even creme, special module only has some models]

Conclusion: Preliminary tests along these lines are not promising, as accuracy scores are in the 52-53% range which is quite bad.
It is possible that introdusing different features may help or that this method is more sensitive to tuning of hyper-parameters, 
so may revisit later. Pischedda obtained much better results (59-60% accuracy) using a similar method so should do better



In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, f1_score


In [2]:
#this takes the odds eg -200 is the favorite, 140 is underdog and says fav wins 

def fav_win(x):
    if x <=0:
        return 1
    if x>0:
        return 0
    
v_fav_win = np.vectorize(fav_win)


def make_win(x):
    if x <= 0:
        return 0
    if x >0:
        return 1

v_make_win = np.vectorize(make_win)


In [3]:
data_dic = {}

file_path_12 = '/Users/joejohns/data_bootcamp/GitHub/nhl_prediction_Aug_2021B/Data/Processed_Data/Approach_1 and_2_win_loss_and_cumul_1seas_Pisch_data/shaped_Pisch_data_xg_added/data_dummies_Pis_v2_20122013.csv'
data_dic[20122013] = pd.read_csv(file_path_12)

for season in [20152016, 20162017, 20172018, 20182019]:   
    
    file_path_seas = '/Users/joejohns/data_bootcamp/GitHub/nhl_prediction_Aug_2021B/Data/Processed_Data/Approach_1 and_2_win_loss_and_cumul_1seas_Pisch_data/shaped_Pisch_data_xg_added/data_dummies_Pis_xg_Corsi_v3_'+str(season)+'.csv'
    data_dic[season] = pd.read_csv(file_path_seas)
 # data_bootcamp/GitHub/final_project_nhl_prediction/Note_books/Explore_Models/data_dummies_Pis_v2_20122013.csv
#data_bootcamp/GitHub/final_project_nhl_prediction/Note_books/Explore_Models/data_dummies_Pis_xg_Corsi_v3_20152016.csv

##probably better to just do all seasons in one big file ... can do that later

all_seasons = sorted(set(data['season']))
all_seasons

X_dic = {}
y_dic = {}
for sea in all_seasons:
   X_dic[sea] = make_HA_data(data, sea)[0]
   y_dic[sea] = make_HA_data(data, sea)[1]


In [4]:
#this is for  regressors predicting wins - losses, can use this to turn output into win prediction 

def make_win(x):
    if x <= 0:
        return 0
    if x >0:
        return 1

v_make_win = np.vectorize(make_win)

#useage: v_make_win(y_pred)

In [5]:
k =5
for season in [20122013,20152016, 20162017, 20172018, 20182019]:    
    filter = (data_dic[season]['full_date'] >= data_dic[season]['full_date'][0]+k).copy() #removes first k = 5 days of season where there are nan values 
    data_dic[season]= data_dic[season].loc[filter, :].copy()



In [6]:
data_12 = data_dic[20122013].copy()  
data_15 = data_dic[20152016].copy()
data_16  = data_dic[20162017].copy()
data_17 = data_dic[20172018].copy()
data_18 = data_dic[20182019].copy()
data_15_17 = pd.concat([data_15,data_16])
data_17_19 = pd.concat([data_17,data_18])
#Note Bene
data_12.rename(columns ={'win%':'win%_cumul'}, inplace = True)
data_12.rename(columns ={'last_10_games_win%' :'win%_last_10_games'}, inplace = True)
#(1230, 50)
#(1230, 50)
#(1271, 51) Vegas, baby
#(1271, 51)'win%_last_10_games'

In [7]:
columns_target = list(data_15.iloc[:5, :8].columns)
data_15.iloc[:5, :8].columns

Index(['Unnamed: 0', 'game_id', 'full_date', 'home_team', 'away_team', 'Open',
       'goal_difference', 'won'],
      dtype='object')

In [8]:

X_12 = data_12.iloc[:,7:].copy() 
X_15 = data_15.drop(columns = columns_target ).copy()
X_16 = data_16.drop(columns = columns_target ).copy()
X_17 = data_17.drop(columns = columns_target ).copy()
X_18 = data_18.drop(columns = columns_target ).copy()
y_12 = data_12.iloc[:,:7].copy() 
y_15 = data_15.loc[:, columns_target ].copy()
y_16 = data_16.loc[:, columns_target ].copy()
y_17 = data_17.loc[:, columns_target ].copy()
y_18 = data_18.loc[:, columns_target ].copy()
list_X = [X_12, X_15,X_16, X_17, X_18]
list_y = [y_12, y_15, y_16,y_17, y_18 ]
list_Xy = zip(list_X, list_y)
list_data = [data_12, data_15, data_16, data_17, data_18]

In [10]:
##note KNN or other clusters might be helpful group the teams in smart way ... but not now.
#models

##regression
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

#classifiers (non-tree)
from sklearn.linear_model import RidgeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression, SGDRegressor, SGDClassifier
from sklearn.svm import SVC


#tree-based classifiers
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from xgboost import XGBRegressor

In [11]:

##regression models
lr = Ridge(alpha=0.001) 
rfr = RandomForestRegressor(max_depth=3, random_state=0)
xgbr = XGBRegressor()
sgdr = SGDRegressor() #has partial_fit()

##classifier models
lrc = RidgeClassifier()
gnb = GaussianNB()  #has partial_fit()
lgr = LogisticRegression(random_state = 0, max_iter = 10**5)
svc = SVC()
sgdc = SGDClassifier() #has partial_fit()

#tree-based classifiers
rfc =  RandomForestClassifier(max_depth=3, random_state=0)
bc = BaggingClassifier()
gbc = GradientBoostingClassifier()
xgbc = XGBClassifier(use_label_encoder=False)

##have partial_fit()


#['BernoulliNB', 'GaussianNB', 'MiniBatchKMeans', 'MultinomialNB', 'PassiveAggressiveClassifier', PassiveAggressiveRegressor', 'Perceptron', 'SGDClassifier', 'SGDRegressor']

In [12]:

from sklearn.preprocessing import StandardScaler, MinMaxScaler 
std_scal = StandardScaler()
mm_scal = MinMaxScaler()

In [13]:
columns_to_scale = list(data_15.iloc[:5, 38:].columns)
data_15.iloc[:5, 38:].columns

Index(['goalsAgainst_cumul_sum', 'goalsFor_cumul_sum', 'goalsDiff_cumul_sum',
       'goalsFor%_cumul_avg', 'pp%_cumul_avg', 'pk%_cumul_avg',
       'sh%_cumul_avg', 'sv%_cumul_avg', 'PDO_cumul_avg',
       'fenwickPercentage_cumul_avg', 'win%_last_10_games', 'win%_cumul'],
      dtype='object')

In [14]:
columns_target = list(data_15.iloc[:5, :8].columns)
data_15.iloc[:5, :8].columns

Index(['Unnamed: 0', 'game_id', 'full_date', 'home_team', 'away_team', 'Open',
       'goal_difference', 'won'],
      dtype='object')

In [15]:
#experimenting with std_scaler ... 
#NOte Bene ... you must assign the numpy object std_scal.fit_transform(Y.loc[:,columns_to_scale]).copy()
#to the df ... if you assign pd.DataFrame(same) it will give NaNs (why?)
 
Y = X_12.iloc[:300, :].copy()
#Y.loc[:,columns_to_scale] = pd.DataFrame(std_scal.fit_transform(Y.loc[:,columns_to_scale])).copy()
Y.loc[:,columns_to_scale] = std_scal.fit_transform(Y.loc[:,columns_to_scale]).copy()
Y.loc[:,columns_to_scale]
Y2 = X_12.iloc[300:, :].copy()
Y2.loc[:,columns_to_scale] = std_scal.transform(Y2.loc[:,columns_to_scale])
#X_12.iloc[300:, :]

#are these two scalars independent? yes ... but this doesn't work if you do std_scal = Sta ...  and use that ... bec std_scal is one instance
scal = StandardScaler()
scal_T = StandardScaler()


Y = X_12.iloc[:300, :].copy()
Y_T = X_12.iloc[300:, :].copy()

#Y.loc[:,columns_to_scale] = pd.DataFrame(std_scal.fit_transform(Y.loc[:,columns_to_scale])).copy()
#scal.fitY.loc[:,columns_to_scale])
scal.fit(Y.loc[:,columns_to_scale])
scal_T.fit(Y_T.loc[:,columns_to_scale])

scal.transform(Y.loc[:,columns_to_scale])


array([[-0.02346027,  0.27487007,  0.21805609, ..., -0.45399568,
         0.5798809 ,  0.62311823],
       [-0.12258815, -0.62634327, -0.35220268, ..., -0.23861609,
        -1.21075221, -1.30087212],
       [ 0.67043491, -0.06308493, -0.5965993 , ..., -0.971329  ,
        -1.80762991, -1.94220223],
       ...,
       [-0.41997179,  0.61282507,  0.78831487, ..., -0.67734162,
         0.34112982,  0.80635541],
       [-1.60950638, -1.75285995,  0.05512502, ...,  1.5316141 ,
        -0.37512342,  0.27038667],
       [-1.41125061, -0.51369161,  0.78831487, ...,  1.59684997,
        -0.73325005,  0.21183044]])

Next cell we try 2015-16 and all models ... training up to T = 800 and predicting d =100 (vary these T,d ... see if anything is good)

In [17]:
T = 800  #train until this game in season (out of 1200)
d = 100  #predict this many games 
scal = std_scal
#X_15 has about 1200 games

#set the data frame and target
X = X_15.copy()
y = y_15.loc[:, 'won'].copy()

#how does it predict on next season?? Terrible! lol ... train around 80% test 47%
#W = X_16.copy()
#z = y_16.loc[:, 'won'].copy()

for model in [lrc,  gnb, lgr, svc,  rfc, bc, gbc, xgbc]:
    y_train= y.iloc[:T].copy()
    y_test = y.iloc[T:T+d].copy()
    #y_test = z.iloc[:100].copy()

    X_train = X.iloc[:T, :].copy()
    X_test= X.iloc[T:T+d, :].copy()   
    #X_test = W.iloc[:100].copy()
    
    #do standard/minmax scaling on X_train numeric columns ... better to do pipeline? 
    #X_train.loc[:, columns_to_scale] = scal.fit_transform(X_train.loc[:, columns_to_scale]).copy()
    
    #fit the scaler from train portion to the test portion 
    #X_test.loc[:, columns_to_scale] = scal.transform(X_test.loc[:, columns_to_scale]).copy()
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
  
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test,y_pred)
    
    print(model, ' TEST accuracy ', acc, 'test f1', f1)
    
    y_pred_train_err =  model.predict(X_train) #! careful with this code
    f1_train_err = f1_score(y_train,y_pred_train_err)
    acc_train_err = accuracy_score(y_train, y_pred_train_err)
    #print(' training error ', model, acc_train_err, ) #f1_train_err)
   
   
#I am following this stackexchange
#from sklearn.preprocessing import MinMaxScaler
#In [93]: mms = MinMaxScaler()
#In [94]: df[['x','z']] = mms.fit_transform(df[['x','z']])
#the one with check mark does pipe line tho :-) https://stackoverflow.com/questions/43834242



RidgeClassifier()  TEST accuracy  0.51 test f1 0.588235294117647
GaussianNB()  TEST accuracy  0.53 test f1 0.591304347826087
LogisticRegression(max_iter=100000, random_state=0)  TEST accuracy  0.46 test f1 0.55
SVC()  TEST accuracy  0.51 test f1 0.6141732283464567
RandomForestClassifier(max_depth=3, random_state=0)  TEST accuracy  0.53 test f1 0.6802721088435373
BaggingClassifier()  TEST accuracy  0.44 test f1 0.4166666666666667
GradientBoostingClassifier()  TEST accuracy  0.56 test f1 0.6206896551724138
[17:18:34] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=

Next we try varying d, for a few fixed models

In [18]:
#T = 800  #train until this game in season (out of 1200)
d = 20  #predict this many games 
scal = StandardScaler() 
#X_15 has about 1200 games

#set the data frame and target
X = X_15.copy()
y = y_15.loc[:, 'won'].copy()

model = gnb
#model = sgdc
##quick checks 

counter = 0
acc_sum_train = 0
acc_sum_test = 0
#for model in [gnb, sgdc]:  #partial_fit
for T in range(d, 800,d):
    y_train= y.iloc[:T].copy()
    y_test = y.iloc[T:T+d].copy()
    #y_test = z.iloc[:100].copy()

    X_train = X.iloc[:T, :].copy()
    X_test= X.iloc[T:T+d, :].copy()   
    #X_test = W.iloc[:100].copy()
    
    #do standard/minmax scaling on X_train numeric columns ... better to do pipeline? 
    X_train.loc[:, columns_to_scale] = scal.fit_transform(X_train.loc[:, columns_to_scale]).copy()
    
    #fit the scaler from train portion to the test portion 
    X_test.loc[:, columns_to_scale] = scal.transform(X_test.loc[:, columns_to_scale]).copy()
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
  
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test,y_pred)
    
    y_pred_train_err =  model.predict(X_train) #! careful with this code
    f1_train_err = f1_score(y_train,y_pred_train_err)
    acc_train_err = accuracy_score(y_train, y_pred_train_err)
    #print(' training error ', model, acc_train_err, ) #f1_train_err)
   
    
    #print(model, ' train error ', acc_train_err, ' f1_train ', f1_train_err, ' TEST ERROR ', acc, ' f1 ', f1)
    acc_sum_test += acc
    acc_sum_train += acc_train_err
    counter +=1
avg_acc = acc_sum_test/counter
avg_acc_train = acc_sum_train/counter
print(model, 'NOO partial_fit' ' avg train error ', avg_acc_train, ' AVERAGE TEST ERROR ', avg_acc,)
   

GaussianNB() NOO partial_fit avg train error  0.6732921003851697  AVERAGE TEST ERROR  0.5282051282051281


In [24]:
#### 
D= 1000 #train until this game in season (out of 1200)
d = 2  #predict this many games 
scal = StandardScaler() 
#X_15 has about 1200 games

#set the data frame and target
X = X_15.copy()
y = y_15.loc[:, 'won'].copy()

#model_np = GaussianNB()
#model= GaussianNB()
model_np = SGDClassifier()
##quick checks 

counter = 0
acc_sum_train_np = 0
acc_sum_test_np = 0
#for model in [gnb, sgdc]:  #partial_fit
for T in range(d, D,d):
    y_train= y.iloc[:T].copy()
    y_test = y.iloc[T:T+d].copy()
    #y_test = z.iloc[:100].copy()

    X_train = X.iloc[:T, :].copy()
    X_test= X.iloc[T:T+d, :].copy()   
    #X_test = W.iloc[:100].copy()
    
    #do standard/minmax scaling on X_train numeric columns ... better to do pipeline? 
    X_train.loc[:, columns_to_scale] = scal.fit_transform(X_train.loc[:, columns_to_scale]).copy()
    
    #fit the scaler from train portion to the test portion 
    X_test.loc[:, columns_to_scale] = scal.transform(X_test.loc[:, columns_to_scale]).copy()
    
    model_np.fit(X_train, y_train)
    y_pred = model_np.predict(X_test)
  
    acc = accuracy_score(y_test, y_pred)
    #f1 = f1_score(y_test,y_pred)
    
    y_pred_train_err =  model_np.predict(X_train) #! careful with this code
   # f1_train_err = f1_score(y_train,y_pred_train_err)
    acc_train_err = accuracy_score(y_train, y_pred_train_err)
    #print(' training error ', model, acc_train_err, ) #f1_train_err)
   
    
    #print(model, ' train error ', acc_train_err, ' f1_train ', f1_train_err, ' TEST ERROR ', acc, ' f1 ', f1)
    acc_sum_test_np += acc
    acc_sum_train_np += acc_train_err
    counter +=1
avg_acc_np = acc_sum_test_np/counter
avg_acc_train_np = acc_sum_train_np/counter
print(model, 'NOO partial_fit' ' avg train error ', avg_acc_train_np, ' AVERAGE TEST ERROR ', avg_acc_np,)
   

#T = 800  #train until this game in season (out of 1200)
#d = 20  #predict this many games 

#two independent scalers, one for <=T, one for T-d, T
scal = StandardScaler()
scal_T = StandardScaler()

#X_15 has about 1200 games

#set the data frame and target
#X = X_16.copy()
#y = y_16.loc[:, 'won'].copy()

#model = gnb
#model = sgdc
##quick checks 

#for model in [gnb, sgdc]:  #partial_fit
counter = 0
acc_sum_train = 0
acc_sum_test = 0
acc_sum_test_T = 0
for T in range(d, D ,d):
    y_train= y.iloc[T-d:T].copy()
    y_test = y.iloc[T:T+d].copy()
    #y_test = z.iloc[:100].copy()

    X_train = X.iloc[T-d:T, :].copy()
    X_scaling_T = X.iloc[:T, :].copy()  #use this to scale the test data ... if we use T-d to T scaler will fluctuate a lot.
    X_test= X.iloc[T:T+d, :].copy() 
    X_test_T= X.iloc[T:T+d, :].copy() 
    #X_test = W.iloc[:100].copy()
    
    #do standard/minmax scaling on X_train numeric columns ... better to do pipeline? 
    X_train.loc[:, columns_to_scale] = scal.fit_transform(X_train.loc[:, columns_to_scale]).copy()
    
    #fit the scaler on all < = T and use this to transform [T to T+d] (and try not as well)
    scal_T.fit(X_scaling_T.loc[:, columns_to_scale])
    X_test_T.loc[:, columns_to_scale] = scal_T.transform(X_test_T.loc[:, columns_to_scale]).copy()
    X_test.loc[:, columns_to_scale] = scal.transform(X_test.loc[:, columns_to_scale]).copy()
    
    model.partial_fit(X_train, y_train, classes=np.unique([1,0]))
    y_pred_T = model.predict(X_test_T)
    y_pred = model.predict(X_test)
  
    acc_T = accuracy_score(y_test, y_pred_T)
    acc= accuracy_score(y_test, y_pred)
    #f1 = f1_score(y_test,y_pred)
    
    y_pred_train_err =  model.predict(X_train) #! careful with this code
    #f1_train_err = f1_score(y_train,y_pred_train_err)
    acc_train_err = accuracy_score(y_train, y_pred_train_err)
    #print(' training error ', model, acc_train_err, ) #f1_train_err)
   
    
    #print(model, 'with partial_fit ', ' train error ', acc_train_err, ' f1_train ', f1_train_err, ' TEST ERROR ', acc, ' f1 ', f1)
    acc_sum_test += acc
    acc_sum_test_T += acc_T
    acc_sum_train += acc_train_err
    counter +=1
avg_acc = acc_sum_test/counter
avg_acc_T = acc_sum_test_T/counter
avg_acc_train = acc_sum_train/counter
#dic[D] = [D, avg_acc_train,  avg_acc, avg_acc_T, avg_acc_np]
#print(dic[D])
print(model, 'with partial_fit ', ' avg train accuracy ', avg_acc_train, ' AVERAGE TEST accuracy (acc, acc_T) ', avg_acc, avg_acc_T)

SGDClassifier() NOO partial_fit avg train error  0.6330287399645583  AVERAGE TEST ERROR  0.5120240480961924
SGDClassifier() with partial_fit   avg train accuracy  0.8557114228456913  AVERAGE TEST accuracy (acc, acc_T)  0.5190380761523046 0.5280561122244489
